请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

# 项目介绍
本工程以分割几种常见的任务为例，学习使用PaddleSeg

项目一：本项目以眼底数据集为例，学习使用PaddleSeg。

项目二：

# 第一步 PaddleSeg安装

环境安装参考链接：
https://github.com/PaddlePaddle/PaddleSeg/blob/release/2.4/docs/install_cn.md

 第1步 安装PaddlePaddle，并测试安装成功。

 第2步 安装PaddleSeg，并测试安装成功。
 
  `[疑问`为什么已经pip装了PaddleSeg,还需要下载源码安装各种依赖呢？不应该pip安装的时候会先自动安装相应依赖吗？


In [1]:
# 安装PaddlePaddle
! python -m pip install paddlepaddle-gpu==2.2.1.post101 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
# 测试安装成功
import paddle
paddle.utils.run_check()

# 安装PaddleSeg
! pip install paddleseg
! git clone https://github.com/PaddlePaddle/PaddleSeg
! cd PaddleSeg
! pip install -r requirements.txt
! python predict.py \
       --config configs/quick_start/bisenet_optic_disc_512x512_1k.yml \
       --model_path https://bj.bcebos.com/paddleseg/dygraph/optic_disc/bisenet_optic_disc_512x512_1k/model.pdparams\
       --image_path docs/images/optic_test_image.jpg \
       --save_dir output/result

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in links: https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
     |████████████████████████████████| 433.9 MB 32 kB/s               
  Attempting uninstall: paddlepaddle-gpu
    Found existing installation: paddlepaddle-gpu 2.2.2.post101
    Uninstalling paddlepaddle-gpu-2.2.2.post101:
      Successfully uninstalled paddlepaddle-gpu-2.2.2.post101
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.
Running verify PaddlePaddle program ... 


W0201 19:36:31.418609   149 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0201 19:36:31.422530   149 device_context.cc:465] device: 0, cuDNN Version: 7.6.


PaddlePaddle works well on 1 GPU.
PaddlePaddle works well on 1 GPUs.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 275 kB 9.4 MB/s            
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.
正克隆到 'PaddleSeg'...
remote: Enumerating objects: 16061, done.
remote: Counting objects: 100% (842/842), done.
remote: Compressing objects: 100% (514/514), done.
^C对象中:  21% (3441/16061), 14.41 MiB | 27.00 KiB/s     
/bin/bash: 第 0 行: cd: PaddleSeg: 没有那个文件或目录
ERROR: Could not open requirements file: [Errno 2] 没有那个文件或目录: 'requirements.txt'
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.
python: can't open file 'predict.py': [Errno 2] No such file or directory


# 第二步 下载数据后并准备训练数据集

In [3]:
# 生成图像列表
import os

path_origin = 'FundusVessels/JPEGImages/'
path_seg = 'FundusVessels/Annotations/'
pic_dir = os.listdir(path_origin)

f_train = open('train_list.txt', 'w')
f_val = open('val_list.txt', 'w')

for i in range(len(pic_dir)):
    if pic_dir[i][-4:]!=".jpg": # 防止有其他隐藏文件
        continue
    if i % 30 != 0:
        f_train.write(path_origin + pic_dir[i] + ' ' + path_seg + pic_dir[i].split('.')[0] + '.png' + '\n')
    else:
        f_val.write(path_origin + pic_dir[i] + ' ' + path_seg + pic_dir[i].split('.')[0] + '.png' + '\n')

f_train.close()
f_val.close()

In [ ]:
!python pretrained_model/download_model.py unet_bn_coco

# 第三步 准备训练代码

参考链接：https://github.com/PaddlePaddle/PaddleSeg/blob/release/2.4/docs/train/train_cn.md

使用quick_start里面的网络结构进行训练，网络是BiSeNetV2网络。

# 第四步 训练、测试网络

训练过程及结果记录：

#### Bisenet网络，轻量化网络
#结论：尝试调参，达不到性能指标
1.无脑直接训练：

batchsize 4，inter：2000：mIoU (0.6595)

2.从数据和硬件角度，设置适合的batchsize：
train_data:193; val_data:7；
193/16 = 12；

batchsize 16，inter 800:mIoU(0.6347) 

3.修改训练图像输入微原始训练图像输入，再训练。（其他超参数和2保持一致）
batchsize 16，inter 800:mIoU(0.6373)

4.batchsize 16，inter 2000:mIoU(0.6671)（0.66*后就不变了）

5.batchsize 64，inter 2000:mIoU(0.6657)（0.66*后就不变了）

#### U2net网络

分分钟满足作业指标：

mIoU (0.7885)

5.应该选一个大网络去处理这个问题，应该很快就能满足mIoU>0.7的目标。

4应该画下训练loss曲线，方便调整超参数。在aistudio上使用visualDL，本机上看不到127.0.0.1的信息。需要把log信息download到本地，然后使用visualDL查看吗？





一个epoch需要12次
12*25=300
12*40=480


In [5]:
# 训练的yaml文件
# unet_optic.yaml
# 训练代码使用PaddleSeg下的train.py文件
!export CUDA_VISIBLE_DEVICES=0
!pwd
!python PaddleSeg/train.py\
--config PaddleSeg/configs/quick_start/yandishuju_bisenet_optic.yml \
--batch_size 4 \
--iters 2000\
--learning_rate 0.01\
--save_interval 200\
--save_dir PaddleSeg/output\
--seed 2021\
--log_iters 20\
--do_eval\
--use_vdl

/home/aistudio
2022-02-03 15:36:38 [INFO]	
------------Environment Information-------------
platform: Linux-4.13.0-36-generic-x86_64-with-debian-stretch-sid
Python: 3.7.4 (default, Aug 13 2019, 20:35:49) [GCC 7.3.0]
Paddle compiled with cuda: True
NVCC: Cuda compilation tools, release 10.1, V10.1.243
cudnn: 7.6
GPUs used: 1
CUDA_VISIBLE_DEVICES: None
GPU: ['GPU 0: Tesla V100-SXM2-16GB']
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~16.04) 7.5.0
PaddleSeg: 2.4.0
PaddlePaddle: 2.2.2
OpenCV: 4.1.1
------------------------------------------------
leiqing: train_dataset: <paddleseg.datasets.dataset.Dataset object at 0x7f943a990f90>
2022-02-03 15:36:38 [INFO]	
---------------Config Information---------------
batch_size: 4
iters: 2000
loss:
  coef:
  - 1
  - 1
  - 1
  - 1
  - 1
  - 1
  - 1
  types:
  - ignore_index: 255
    type: CrossEntropyLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.01
  power: 0.9
  type: PolynomialDecay
model:
  num_classes: 2
  pretrained: null
  type: U2Net
optimizer:
  momentu

In [6]:
# 单独进行评估 -- 上边do_eval就是这个工作
!python PaddleSeg/val.py\
--config PaddleSeg/configs/quick_start/yandishuju_bisenet_optic.yml \
--model_path PaddleSeg/output/best_model/model.pdparams

2022-02-03 16:03:39 [INFO]	
---------------Config Information---------------
batch_size: 4
iters: 1000
loss:
  coef:
  - 1
  - 1
  - 1
  - 1
  - 1
  - 1
  - 1
  types:
  - type: CrossEntropyLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.01
  power: 0.9
  type: PolynomialDecay
model:
  num_classes: 2
  pretrained: null
  type: U2Net
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-05
train_dataset:
  dataset_root: PaddleSeg/data
  mode: train
  num_classes: 2
  train_path: PaddleSeg/data/FundusVessels/train_list.txt
  transforms:
  - target_size:
    - 565
    - 548
    type: Resize
  - type: RandomHorizontalFlip
  - type: Normalize
  type: Dataset
val_dataset:
  dataset_root: PaddleSeg/data
  mode: val
  num_classes: 2
  transforms:
  - type: Normalize
  type: Dataset
  val_path: PaddleSeg/data/FundusVessels/val_list.txt
------------------------------------------------
W0203 16:03:39.470232  2841 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, 

In [12]:
# 那一张测图像，查看分割效果
!python PaddleSeg/predict.py\
--config PaddleSeg/configs/quick_start/yandishuju_bisenet_optic.yml\
--model_path PaddleSeg/output/best_model/model.pdparams\
--image_path PaddleSeg/data/FundusVessels/JPEGImages/ \
--save_dir PaddleSeg/output/FundusVessels/result

2022-02-03 16:11:52 [INFO]	
---------------Config Information---------------
batch_size: 4
iters: 1000
loss:
  coef:
  - 1
  - 1
  - 1
  - 1
  - 1
  - 1
  - 1
  types:
  - type: CrossEntropyLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.01
  power: 0.9
  type: PolynomialDecay
model:
  num_classes: 2
  pretrained: null
  type: U2Net
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-05
train_dataset:
  dataset_root: PaddleSeg/data
  mode: train
  num_classes: 2
  train_path: PaddleSeg/data/FundusVessels/train_list.txt
  transforms:
  - target_size:
    - 565
    - 548
    type: Resize
  - type: RandomHorizontalFlip
  - type: Normalize
  type: Dataset
val_dataset:
  dataset_root: PaddleSeg/data
  mode: val
  num_classes: 2
  transforms:
  - type: Normalize
  type: Dataset
  val_path: PaddleSeg/data/FundusVessels/val_list.txt
------------------------------------------------
W0203 16:11:52.829084  3749 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, 